In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import wandb as wb
from src.MetaHeauristic import WandbHandler, OBP
import re, logging
import time
import json

In [2]:
wb.login()

wandb: Currently logged in as: glasshan914 (glasshan9140) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# def get_sample(dataset)->pd.DataFrame:
#     total_order_num=len(dataset.order_num.unique())
#     #파일 중 일부 order만 샘플링 후 수행
#     sample_order_num=np.random.random_integers(1,total_order_num,1) #N: 몇개의 order를 뽑을 것인지
#     sample_order=np.random.choice(total_order_num,size=sample_order_num,replace=False) #선택된 order_id
#     sample1=dataset[dataset['order_num'].isin(sample_order)] 
#     print(f'{f}번째 파일, 주문 수: {sample_order_num}')
#     print(f'주문 id: {sample_order}')
#     return sample1, f, sample_order_num, sample_order


# folder_list=glob.glob('E:/glass_git/AI-portfolio/order_batch_picking/data/Orderlinelist/*.txt')
# dist_matrix=pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')
# dist_matrix=dist_matrix.to_numpy()

# datasets=[]
# for file in folder_list:
#     df=pd.read_csv(file,sep='\t',header=None)
#     cols=['order_num','index','rack','bay','tier','slot']
#     df.columns=cols
#     datasets.append(df)
# print(f'total files: {len(datasets)}')

# #여러개 파일 중에 N번째 파일 선택
# f=np.random.random_integers(1,len(datasets),1).item() #몇번째 파일을 선택할 것인지
# dataset=datasets[f]

# #데이터 샘플링
# sample,f,sample_order_num,sample_order=get_sample(dataset)

In [4]:
# logger = logging.getLogger("mealpy")
# logger.setLevel(logging.INFO)
# logger.propagate = False  # 중복 출력 방지 (필요 시)
# if not any(isinstance(h, WandbHandler) for h in logger.handlers):
#     logger.addHandler(WandbHandler())

# sample=pd.read_csv('E:\glass_git\AI-portfolio\order_batch_picking\data\Order\episode1_orders.csv')
# location=pd.read_csv('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/location.csv')
# dist_matrix=pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')
# dist_matrix=dist_matrix.to_numpy()

# try_id=time.strftime('%y%m%d_%H%M')
# K = 10
# ALPHA = 0.8
# model_params={'epoch':1000 , 'popsize': 100, 'nlimits': 3 , 'seed': 42} 

# wbrun=wb.init(project="OBP", name=f"abc-{try_id}", 
#         config=dict(
#             algo="ABC", 
#             epoch=model_params.get('epoch'), 
#             pop_size=model_params.get('popsize'), 
#             n_limits=model_params.get('nlimits'), 
#             seed=model_params.get('seed'),s
#             data_file_num='episode1',
#             the_number_of_orders=len(sample),
#             orders_id=sample['order_num'].unique().tolist(),
#             job=K,
#             alpha=ALPHA,
#             depot=0
# ))

# with wbrun:
#     obp=OBP(dist_matrix,sample,location, K,ALPHA)
#     model=obp.model(model_params)
#     best=obp.run(model)

In [ ]:
logger = logging.getLogger("mealpy")
logger.setLevel(logging.INFO)
logger.propagate = False  # 중복 출력 방지 (필요 시)
if not any(isinstance(h, WandbHandler) for h in logger.handlers):
    logger.addHandler(WandbHandler())

sample=pd.read_csv('E:\glass_git\AI-portfolio\order_batch_picking\data\Order\episode1_orders.csv')
location=pd.read_csv('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/location.csv')
dist_matrix=pd.read_pickle('E:/glass_git/AI-portfolio/order_batch_picking/data/Distance matrix/dist_matrix.pkl')
dist_matrix=dist_matrix.to_numpy()

try_id=time.strftime('%y%m%d_%H%M')
K = 4
ALPHA = 0.8
model_params={'epoch':500 , 'popsize': 100, 'nlimits': 3 , 'seed': 42} 


Model_list=['SPO','HHO','ABC','WaOA','ServalOA','STO'] #,

for model_name in Model_list:
    try_id=time.strftime('%y%m%d_%H%M')

    wbrun=wb.init(project="OBP", name=f"{model_name}-{try_id}", 
            config=dict(
                algo=model_name, 
                epoch=model_params.get('epoch'), 
                pop_size=model_params.get('popsize'), 
                n_limits=model_params.get('nlimits'), 
                seed=model_params.get('seed'),
                data_file_num='episode1',
                the_number_of_orders=len(sample),
                orders_id=sample['order_num'].unique().tolist(),
                job=K,
                alpha=ALPHA,
                depot=0
    ))
    artifact = wb.Artifact("best_solution", type="dataset")
    with wbrun:
        obp=OBP(model_name,dist_matrix,sample,location, K,ALPHA)
        model=obp.model(model_params)
        best=obp.run(model)
    
    print(f'best solution: {best.solution}, best fitness: {best.target.fitness}')
    with open("best_solution.json", "w") as f:
        json.dump(best.solution, f)
    artifact.add_file("best_solution.json",name=f'{wbrun.name}_best_solution.json')
    artifact.save()
    save_path=f'E:/glass_git/AI-portfolio/order_batch_picking/result/{wbrun.name}.png'
    obp.result(best,location,save_path)
wbrun.finish()

<>:7: SyntaxWarning: invalid escape sequence '\g'
<>:7: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30056\2288080359.py:7: SyntaxWarning: invalid escape sequence '\g'
  sample=pd.read_csv('E:\glass_git\AI-portfolio\order_batch_picking\data\Order\episode1_orders.csv')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30056\2288080359.py:7: SyntaxWarning: invalid escape sequence '\g'
  sample=pd.read_csv('E:\glass_git\AI-portfolio\order_batch_picking\data\Order\episode1_orders.csv')


AttributeError: 'Run' object has no attribute 'Artifact'